In [1]:
# Competition Link  :   https://www.kaggle.com/competitions/playground-series-s3e16/data

# Evaluation : Mean Absolute Error (MAE)

In [2]:
import pandas as pd
import numpy as np 
from ydata_profiling import ProfileReport

In [5]:
data = pd.read_csv('data/train.csv')

def Overview (data) :
    profile = ProfileReport( data)
    profile.to_file('output.html')


# Overview(data)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
data.shape

(74051, 10)

In [7]:
data.describe()

,id,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
count,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000
mean,37025.000000,1.317460,1.024496,0.348089,23.385217,10.104270,5.058386,6.723870,9.967806
std,21376.826729,0.287757,0.237396,0.092034,12.648153,5.618025,2.792729,3.584372,3.175189
min,0.000000,0.187500,0.137500,0.000000,0.056699,0.028349,0.042524,0.042524,1.000000
25%,18512.500000,1.150000,0.887500,0.300000,13.437663,5.712424,2.863300,3.968930,8.000000
50%,37025.000000,1.375000,1.075000,0.362500,23.799405,9.908150,4.989512,6.931453,10.000000
75%,55537.500000,1.537500,1.200000,0.412500,32.162508,14.033003,6.988152,9.071840,11.000000
max,74050.000000,2.012815,1.612500,2.825000,80.101512,42.184056,21.545620,28.491248,29.000000


In [8]:
# As Univarient analysis is perform in Profile Report so we foucs on multivarient Data Analysis

In [9]:
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px
%matplotlib inline

In [10]:
px.scatter(data, x='Age', y='Length', color='Sex')

In [11]:
px.histogram(data, x='Age', color='Sex')

In [12]:
# from this graph we can clearly see age Sex effect the  averagae of the data